In [ ]:
from dataclasses import dataclass, field
from ctypes import c_uint, c_uint16
from matplotlib import pyplot as plt
import random as rnd

pairs = "..LEXEGEZACEBISO"\
"USESARMAINDIREA."\
"ERATENBERALAVETI"\
"EDORQUANTEISRION"

@dataclass
class seedtype:
    w0: c_uint16
    w1: c_uint16
    w2: c_uint16

@dataclass
class fastseedtype:
    a: c_uint16
    b: c_uint16
    c: c_uint16
    d: c_uint16

@dataclass
class plansys:
    x: c_uint
    y: c_uint
    economy: c_uint
    govtype: c_uint
    techlev: c_uint
    population: c_uint
    productivity: c_uint
    radius: c_uint
    goatsoupseed: fastseedtype
    name: list = field(default_factory=list)


def tweakseed(s: seedtype):
    temp: c_uint16
    temp = c_uint16((s.w0.value) + (s.w1.value) + (s.w2.value))
    s.w0.value = s.w1.value
    s.w1.value = s.w2.value
    s.w2.value = temp.value


def makesystem(s: seedtype) -> plansys:
    thissys: plansys = plansys(c_uint(0), c_uint(0), c_uint(0), c_uint(0), c_uint(0), c_uint(0), c_uint(0), c_uint(0),
        fastseedtype(c_uint16(0),c_uint16(0),c_uint16(0),c_uint16(0)), [0 for i in range(12)]
    )
    pair1: c_uint; pair2:c_uint; pair3:c_uint; pair4: c_uint
    longnameflag: c_uint16
    longnameflag = c_uint16((s.w0.value) & c_uint16(64).value)
    thissys.x.value = (((s).w1.value) >> 8)
    thissys.y.value = (((s).w0.value) >> 8)

    thissys.govtype.value = ((((s).w1.value) >> 3) & 7)

    thissys.economy.value = ((((s).w0.value) >> 8) & 7)
    if (thissys.govtype.value <= 1):
        thissys.economy.value = ((thissys.economy.value) | 2)

    thissys.techlev.value = ((((s).w1.value) >> 8) & 3) + ((thissys.economy.value) ^ 7)
    thissys.techlev.value += ((thissys.govtype.value) >> 1)
    if (((thissys.govtype.value) & 1) == 1):
        thissys.techlev.value += 1

    thissys.population.value = 4 * (thissys.techlev.value) + (thissys.economy.value)
    thissys.population.value += (thissys.govtype.value) + 1

    thissys.productivity.value = (((thissys.economy.value) ^ 7) + 3)*((thissys.govtype.value) + 4)
    thissys.productivity.value *= (thissys.population.value) * 8

    thissys.radius.value = 256 * (((((s).w2.value) >> 8) & 15) + 11) + thissys.x.value

    thissys.goatsoupseed.a.value = (s).w1.value & 0xFF
    thissys.goatsoupseed.b.value = (s).w1.value >> 8
    thissys.goatsoupseed.c.value = (s).w2.value & 0xFF
    thissys.goatsoupseed.d.value = (s).w2.value >> 8

    pair1 = 2 * ((((s).w2.value) >> 8) & 31)
    tweakseed(s)
    pair2 = 2 * ((((s).w2.value) >> 8) & 31)
    tweakseed(s)
    pair3 = 2 * ((((s).w2.value) >> 8) & 31)
    tweakseed(s)
    pair4 = 2 * ((((s).w2.value) >> 8) & 31)
    tweakseed(s)


    (thissys.name)[0] = pairs[pair1]
    (thissys.name)[1] = pairs[pair1 + 1]
    (thissys.name)[2] = pairs[pair2]
    (thissys.name)[3] = pairs[pair2 + 1]
    (thissys.name)[4] = pairs[pair3]
    (thissys.name)[5] = pairs[pair3 + 1]

    if (longnameflag):
        (thissys.name)[6] = pairs[pair4]
        (thissys.name)[7] = pairs[pair4 + 1]
        (thissys.name)[8] = 0
    else:
        (thissys.name)[6] = 0

    return thissys

def cm_to_inch(value):
    return value/2.54

def main():
    fig, ax = plt.subplots(figsize=plt.figaspect(1/2))
    ax.plot([], [])
    ax.set_facecolor((0,0,0))
    seed = seedtype(c_uint16(0x5A4A), c_uint16(0x0248), c_uint16(0xB753))
    for i in range(255):
        system = makesystem(seed)
        ax.scatter(system.x.value, system.y.value, color='white', s=1)
        ax.text(system.x.value + 1, system.y.value + 1, ''.join([x for x in system.name if x != 0 and x != "."]), color="white", fontsize=5)
    plt.gca().invert_yaxis()
    plt.show()

main()